<h1 id="tocheading">Table of Contents</h1>
<div id="toc"></div>

In [3]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

# Useful Links

* Papers
  * An Application of Deep Neural Networks in the Analysis of Stellar Spectra ([Fabbro _et al_, 2018](https://arxiv.org/abs/1709.09182))
  * The Apache Point Observatory Galactic Evolution Experiment (APOGEE) ([Majewski _et al._, 2017](https://arxiv.org/abs/1509.05420))
  * The APOGEE Stellar Parameter and Chemical Abundances Pipeline ([García Pérez _et al._, 2016](https://arxiv.org/abs/1510.07635))
* [APOGEE DR14](https://www.sdss.org/dr14/irspec/)
  * [Overview](https://www.sdss.org/dr14/irspec/), [ASPCAP](https://www.sdss.org/dr14/irspec/aspcap/), [Data Access](https://www.sdss.org/dr14/data_access/), [SkyServer](http://skyserver.sdss.org/dr14/en/home.aspx)
* [APOGEE DR12](https://www.sdss.org/dr12/irspec/)
  * [Overview](https://www.sdss.org/dr12/irspec/), [Data Access](https://www.sdss.org/dr12/data_access/)
* Miscellaneous
  * [StarNet data files](http://apps.canfar.net/storage/list/starnet/public)
  * Public version of these notes on [Github](https://nbviewer.jupyter.org/gist/peterswords/9b8e243995f7fa8842bb3d18ef832f78)

# Notes on running StarNet software

StarNet is a deep neural network approach to deriving stellar parameters from APOGEE spectra (Fabbro _et al._, 2018; O’Briain and Fabbro, 2017). These notes describe my attempts to run the software on Windows and Linux, and document a number of issues encountered. The software itself is written in Python and uses the Keras and TensorFlow machine learning packages. It is [available from github](https://github.com/astroai/starnet) in the form of six IPython/Jupyter Notebooks to be run in turn, with the following functionality:
1. Download data (stellar spectra)
2. Pre-process training data
3. Pre-process test data
4. Train model
5. Test model
6. Propagate errors

The project's `readme` and the notebooks themselves provide good documentation on how to proceed. I don't attempt to replicate that here, rather I just point out some pitfalls.
Note that each of the notebooks processes certain inputs and produces outputs. All of the inputs and outputs are [available online](http://apps.canfar.net/storage/list/starnet/public) from the Canadian Astronomy Data Centre (linked from the first notebook), so any of the steps can be skipped by downloading the appropriate intermediate files. The total data download is ~50 GB -- it is advisable to use a restartable download tool such as `wget` (also available for Windows) as the server can be slow and downloads can be interrupted. You do not need the additional ~25 GB in the `cycle_gan` directory.


## StarNet on different platforms.

#### Software and Hardware Versions
This is where things get complicated. There is no indication which software versions were used to run StarNet, other than a note in the `readme` indicating that the notebooks have "_been tested with python-2.7, some dependencies are not python-3 ready yet_". However, as of now (July 2018) the latest version of TensorFlow (1.8) depends on Python 3, and specifically on a 64-bit version. Versions since 1.6 also use advanced hardware instructions ([AVX vector extensions](https://en.wikipedia.org/wiki/Advanced_Vector_Extensions)) which are not supported on all Intel and AMD CPUs. Matching compatible software and hardware versions turned out to be easier on a modern Windows laptop than an older Linux server. However the Windows performance for training the NN model is (not unexpectedly) considerably worse than the half hour on a modern 16-core Linux machine mentioned in Fabro _et al_. The authors also mention that there are 41,000 observed training spectra each with 7,000 intensity values, 224,000 synthetic training spectra, and ultimately 7.5 million neural network parameters in the trained model. The latest version of Keras [did _not_ work](https://github.com/keras-team/keras/issues/10417) -- the Test notebook would not open the trained model. It was necessary to downgrade from version 2.2.0 of Keras to 2.1.6.

#### Windows
The machine I used is a dual-core i3-5157U at 2.5 GHz, with 8 GB of memory, running Windows 10. I installed Python 3.6 64-bit version from python.org. **Note**: The 32-bit version which installs by default is _not_ compatible with TensorFlow. During the install, check the box to add Python to your path. If prompted at the end of installation, click the button to remove Windows 260 character path limitation.

For a development environment I installed the free Eclipse Oxygen (4.7) IDE from eclipse.org. Within Eclipse I then installed the PyDev environment from pydev.org. Although this development setup is not mandatory I was already familiar with Eclipse, and PyDev comes with the Pip package manager built in which simplifies installation of some Python package dependencies compared to, say, Anaconda.

The Python environment and Pip packages can be configured in Eclipse through `Windows > Preferences > PyDev > Interpreters > Python Interpreter`. If you installed Python before PyDev, it will already have picked up your Python install, otherwise use "Quick Auto-Config". This will show you the installed location of your Python executable, which you will need in order to run Jupyter notebooks. Then use "Install with Pip" to install the package dependencies referenced in the StarNet notebooks. After you have installed the Jupyter package, the location of the Jupyter executable will be based off your Python install location, something like: `C:\Users\SOME_USER\AppData\Local\Programs\Python\Python36\Scripts\jupyter-notebook.exe`.

You can now run the StarNet notebooks. Note that depending on where you placed the downloaded data you may need to modify the `datadir` variable in each notebook. Notebooks 2 and 3 (extracting training and testing data) take about 15 minutes each to run. If you are using downloaded data inputs for Notebook 3 you need to rename `aspcapStar_combined_main_dr12.h5` to `apStar_combined_main.h5`. There is also a line in this notebook that tries to delete any pre-existing output `test_data.h5` which will fail if you are running for the first time.

#### Linux
Here the machine had 8 x quad-core Xeon E5430 CPUs @ 2.66GHz, with 16 GB memory, running Ubuntu 16.04.4 LTS. Python versions 2.7 and 3.5 were pre-installed. Since the training of the model is the most compute intensive part of the process I experimented with this by extracting the Python code from the relevant notebook into a standalone program that could be run at the command line.

After installing the latest Keras and Tensorflow packages I ran the training program. This immediately [core-dumped](https://stackoverflow.com/questions/49094597/illegal-instruction-core-dumped-after-running-import-tensorflow) as the off-the-shelf Tensorflow binaries contain AVX vector instructions which are not supported on these decade-old Harpertown 5400-series processors.

My next attempt was to downgrade from Tensorflow 1.8 to 1.5, the newest version built without AVX instructions. This did not core-dump, but failed nonetheless, reporting an incompatibility with the installed `numpy` Python library. Tensorflow 1.5 declares a minimum version dependency on `numpy >= 1.13`, but the version installed was 1.11. I then tried to roll back Tensorflow to a version with an earlier `numpy` dependency, which required downgrading all the way to version 1.12. This ran, but finished prematurely after many hours, and the results were so different from those on other platforms that it was impossible to have any confidence in them.

The next step was to try to upgrade `numpy`, but this initially failed due to a dependency on Python 3.6. So I installed a local version of Python 3.6.5 (roughly following the process [here](http://thelazylog.com/install-python-as-local-user-on-linux/) which allowed me to install `numpy` 1.14. Tensorflow 1.18 still core-dumped, but version 1.15 without the AVX instructions ran with the later `numpy`. However, performance was still abysmal. I had no way to know if this was purely the result of running the older Tensorflow or if versions after 1.15 had introduced other performance enhancements that I was missing.

So my final effort was to [build Tensorflow from sources](https://www.tensorflow.org/install/install_sources), which would allow it to use whatever instruction set was the most efficient for the machine on which it was installed. This was an awkward 6-hour process which involved also installing other development components. Nevertheless it was successful in producing a locally-built Tensorflow 1.18, running on Python 3.6.5. Disappointingly, there was no performance improvement. The training process took most of 6 hours -- half the performance of a more modern dual-core single processor laptop -- in spite of the process getting 200-400% CPU for the duration. The tentative conclusion is that this cannot be improved without processors that support the newer instruction sets, which anecdotally produce factors of two- to "many"-fold speed up on numerically intensive tasks. However, there is one [anecdotal suggestion](https://askubuntu.com/questions/1021882/poor-performance-in-tensorflow) that TensorFlow simply performs worse on Ubuntu than other operating systems even with identical hardware.

#### Amazon EC2
I rented an [Amazon EC2](https://aws.amazon.com/ec2/pricing/on-demand/) `c5d.4xlarge` compute-optimised instance. This has 16 x octa-core Xeon Platinum 8124M processors @3.0 GHz, for a total of 128 cores, with 32 GB of memory and a 400 GB SSD. The processors are from the 2017 Skylake family, and support `AVX`, `AVX2`, and `AVX-512F` instructions. You *must* launch a so-called Amazon HVM instance to avail of the extended instruction sets.

Some fiddly stuff:
* You have to go through an approval process with Amazon, followed by another to increase your instance limits to allow the high-performance instance. This takes a couple of hours, but just follow the instructions.
* While configuring the instance you are prompted to download a pem file containing authorisation keys for your new instance 
* To connect with `ssh` you must first use Amazon's AWS dashboard to configure inbound traffic to be allowed from your IP address
* To connect with ssh: `ssh -i EC2.pem ec2-user@ec2-52-210-17-192.eu-west-1.compute.amazonaws.com`
  * Replace `EC2.pem` with the full path to pem file
  * `ec2-user` is the default username for Amazon Linux instances
  * Replace `@instanceName` with the instance name from AWS dashboard
  * On windows running `ssh` complains that permissions are too open on the pem file. Instructions for fixing are [here](https://superuser.com/questions/1296024/windows-ssh-permissions-for-private-key-are-too-open).
* `$ cat /proc/version`
  * `Linux version 4.14.47-56.37.amzn1.x86_64 (mockbuild@gobi-build-64011) (gcc version 7.2.1 20170915 (Red Hat 7.2.1-2) (GCC)) #1 SMP Wed Jun 6 18:49:01 UTC 2018`

Installations:
* The Amazon Linux OS uses the `yum` package manager. Python 2.7 is installed by default, but we need 3.6 for tensorflow.
* `sudo yum update`
* `sudo yum list | grep python3`
* `sudo yum install python36.x86_64`
* `sudo python3 -m pip install keras tensorflow`

Running StarNet:
* Download the required data for training using `wget` from the [CADC](http://apps.canfar.net/storage/list/starnet/public) -- `training_data.h5`, `mean_and_std.npy`
* I consolidated the training notebook into a [single Python source](https://pastebin.com/AsM3K93S) for convenience. The raw version can be downloaded with `wget` from pastebin.
* Run it: `python3 Train_Model.py`

Results:
* Training completes in 10 minutes on this configuration
* Tensorflow issues the following warning, suggesting that it might run even faster on a version built from sources to take advantage of extended instructions beyond basic `AVX``:
  * `tensorflow/core/platform/cpu_feature_guard.cc:140. Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA`


## Trial outputs obtained with StarNet

### Tests with abundances in week ending 10-Aug

1. $[\alpha/M]$, $[C/Fe]$, $[N/Fe]$ -- [train](https://nbviewer.jupyter.org/github/peterswords/StarNetTest/blob/master/notebooks/aug09/b1/4_Train_Model.ipynb), [test](https://nbviewer.jupyter.org/github/peterswords/StarNetTest/blob/master/notebooks/aug09/b1/5_Test_Model.ipynb) -- (bad run due to early stopping after 9 epochs)
1. $[\alpha/M]$, $[C/Fe]$, $[N/Fe]$ -- [train](https://nbviewer.jupyter.org/github/peterswords/StarNetTest/blob/master/notebooks/aug09/b2/4_Train_Model.ipynb), [test](https://nbviewer.jupyter.org/github/peterswords/StarNetTest/blob/master/notebooks/aug09/b2/5_Test_Model.ipynb) -- (good run)
1. $T_{eff}$, $[\alpha/M]$, $[C/Fe]$ -- [train](https://nbviewer.jupyter.org/github/peterswords/StarNetTest/blob/master/notebooks/aug09/b3/4_Train_Model.ipynb), [test](https://nbviewer.jupyter.org/github/peterswords/StarNetTest/blob/master/notebooks/aug09/b3/5_Test_Model.ipynb) -- (good run)
1. $T_{eff}$, $[Fe/H]$, $[\alpha/M]$, $[C/Fe]$, $[N/Fe]$ -- [train](https://nbviewer.jupyter.org/github/peterswords/StarNetTest/blob/master/notebooks/aug09/b4/4_Train_Model.ipynb), [test](https://nbviewer.jupyter.org/github/peterswords/StarNetTest/blob/master/notebooks/aug09/b4/5_Test_Model.ipynb) -- (good run)
1. $T_{eff}$, $[Fe/H]$, $[\alpha/M]$, $[C/Fe]$, $[N/Fe]$ -- [train](https://nbviewer.jupyter.org/github/peterswords/StarNetTest/blob/master/notebooks/aug09/b5/4_Train_Model.ipynb), [test](https://nbviewer.jupyter.org/github/peterswords/StarNetTest/blob/master/notebooks/aug09/b5/5_Test_Model.ipynb) -- (filtered $[C/Fe]$, $[N/Fe]$ same as $[Fe/H]$, i.e. must be > -3 dex)





### Good tests in week ending 03-Aug

A series of tests carried out with the help of Teaghan O'Briain in U.Victoria can be found [here](https://nbviewer.jupyter.org/github/peterswords/StarNetTest/blob/master/notebooks/StarNet_compare.ipynb). These successfully replicate the results in Fabbro et. al.

### Bad outputs up to end of July.

Up to end of July all trials were unwittingly using corrupted APOGEE data. Unfortunately the software still ran and reported correct numbers of records, but produced spurious output.

1. This is the original output from the "5_Test_Model" notebook published in Fabro _et al._ 2017. It shows the divergence between values obtained using StarNet and those from the ASPCAP pipeline.<br><br>

<img src="https://i.imgur.com/6NND9L9.png" width="548px" height="430px">
<br><br>

2. This is the output I obtained using the same StarNet software and the input data provided online. Only Python and software library versions have been updated (Python 3.6.5, TensorFlow 1.8, Keras 2.1.6). The results are quite different from those obtained by Fabro _et al_, showing greater divergence from ASPCAP values.<br><br>

<img src="https://i.imgur.com/su3yq3V.png" width="548px" height="430px">
<br><br>

3. Here I have changed the output parameters from $\ T_{eff},\log(g),[Fe/H]\ $ to $\ T_{eff},[C/M],[N/M]\ $. The scatter in $T_{eff}$ is even more pronounced in this version, and the scatter in all values varies systematically with their magnitudes.<br><br>

<img src="https://i.imgur.com/pGvPeSP.jpg" width="548px" height="430px">
<br><br>

4. And here I have output $\ [Fe/H],[C/M], [N/M]$.<br><br>

<img src="https://i.imgur.com/tyuoaUE.png" width="548px" height="430px">

 

# Related Reading

Anything I come across of interest goes in this related reading list, whether I've read it or not. The Index by Subject Area lists titles and authors under various subject headings, with green ticks (<span style="color:green;font-size:18px">✔</span>) indicating the ones I've read. The alphabetical Index by Author contains complete references.

## Index by Subject Area

* **APOGEE-related articles**
  * <span style="color:green;font-size:18px">✔</span> An Application of Deep Neural Networks in the Analysis of Stellar Spectra (Fabbro _et al._, 2018)
  * <span style="color:green;font-size:18px">✔</span> Chemical Cartography with APOGEE: Large-scale Mean Metallicity Maps of the Milky Way Disk (Hayden _et al._, 2014)
  * <span style="color:green;font-size:18px">✔</span> STATUS OF APOGEE DATA PRODUCTS, REDUCTION, AND ANALYSIS (Holtzman _et al._, slideshow)
  * <span style="color:green;font-size:18px">✔</span> The Apache Point Observatory Galactic Evolution Experiment (APOGEE) (Majewski _et al._, 2017)
  * <span style="color:green;font-size:18px">✔</span> ASPCAP: The APOGEE Stellar Parameter and Chemical Abundances Pipeline (García Pérez _et al._, 2016)
  * The APOKASC Catalog: An Asteroseismic and Spectroscopic Joint Survey of Targets in the Kepler Fields (Pinsonneault _et al._, 2014)
  * The First APOKASC Catalog of Kepler Dwarf and Subgiant Stars (Serenelli _et al_ 2017)
  * The Observation and Analysis of Stellar Photospheres, 3rd edn. (Gray, 2008)
* **Software Tools**
  * <span style="color:green;font-size:18px">✔</span> [starnet](https://github.com/astroai/starnet): Stellar spectra features with machine learning (O’Briain & Fabbro, 2018)
  * [Tools for dealing with APOGEE data](https://github.com/jobovy/apogee) (Bovy, 2018)
  * <span style="color:green;font-size:18px">✔</span> [bokeh](https://github.com/bokeh/bokeh): Interactive Web Plotting for Python (Van de Ven & Paprocki, 2018)
* **Machine Learning for Stellar Classification**
  * Stellar parameters from very low resolution spectra and medium band filters: Teff, logg and [M/H] using neural networks (Bailer-Jones, 2000)
  * Physical parametrization of stellar spectra: the neural network approach (Bailer-Jones _et al._, 1997)
  * Automated classification of stellar spectra – I. Initial results with artificial neural networks (von Hippel _et al._, 1994)
  * <span style="color:green;font-size:18px">✔</span> New frontiers in galactic archaeology: spectroscopic surveys, carbon-enhanced metal-poor stars, and machine learning applications (Kielty, 2017)
  * Parameterizing Stellar Spectra Using Deep Neural Networks (Li _et al._, 2017)
  * ANNs and Wavelets: A Strategy for Gaia RVS Low S/N Stellar Spectra Parameterization (Manteiga _et al._, 2010)
  * Stellar spectral classification using principal component analysis and artificial neural networks (Singh _et al._, 1998)
  * A new automated spectral feature extraction method and its application in spectral classification and defective spectra recovery (Wang _et al._, 2017)
* **Machine Learning, General**
  * Machine Learning with Python Cookbook, (Albon, 2018)
  * Python Deep Learning Cookbook (Bakker, 2017)
  * Building Machine Learning Projects with TensorFlow (Bonnin, 2016)
  * Fundamentals of Deep Learning: Designing Next-Generation Machine Intelligence Algorithms (Buduma & Locascio, 2017)
  * Deep Learning with Python (Chollet, 2018)
  * Hands-On Machine Learning with Scikit-Learn and TensorFlow (Géron, 2017)
  * Deep Learning with Keras: Implementing deep learning models and neural networks with the power of Python (Gulli & Pal, 2017)
  * TensorFlow Agents: Efficient Batched Reinforcement Learning in TensorFlow (Hafner, Davidson & Vanhoucke, 2017)
  * Learning TensorFlow: A Guide to Building Deep Learning Systems (Hope, Resheff & Lieder, 2017)
  * Deep Learning with Python: A Hands-on Introduction (Ketkar, 2017)
  * Advanced Data Analytics Using Python: With Machine Learning, Deep Learning and NLP Examples (Mukhopadhyay, 2018)
  * Reinforcement Learning: With Open AI, TensorFlow and Keras Using Python (Nandy & Biswas, 2017)
  * Pro Deep Learning with TensorFlow: A Mathematical Approach to Advanced Artificial Intelligence in Python (Pattanayak, 2017)
  * Shukla, N. (2018) Machine Learning with TensorFlow, 1 edition., Shelter Island, NY, Manning Publications.
  * Machine Learning Solutions (Thanaki, 2018)
  * Deep Learning with TensorFlow: Explore neural networks with Python (Zaccone, Karim & Menshawy, 2017)
  * <span style="color:green;font-size:18px">✔</span> Python Deep Learning (Zocca _et al._, 2017)
* **Other Software References**
  * <span style="color:green;font-size:18px">✔</span> Python in a Nutshell: A Desktop Quick Reference (Martelli, Ravenscroft & Holden, 2017)
  * IPython Interactive Computing and Visualization Cookbook (Rossant, 2018)
  * Jupyter for Data Science: Exploratory analysis, statistical modeling, machine learning, and data visualization with Jupyter (Toomey, 2017)
  * Matplotlib 2.x By Example: Multi-dimensional charts, graphs, and plots in Python (Yu, Chung, & Yim, 2017)
  

## Index by Author

<div class="csl-bib-body" style="line-height: 1.35; ">
  <div class="csl-entry" style="margin-bottom: 1em;">Albon, C. (2018) <i>Machine Learning with Python Cookbook</i>, Sebastopol, O′Reilly.</div>
  <span class="Z3988" title="url_ver=Z39.88-2004&amp;ctx_ver=Z39.88-2004&amp;rfr_id=info%3Asid%2Fzotero.org%3A2&amp;rft_id=urn%3Aisbn%3A978-1-4919-8938-8&amp;rft_val_fmt=info%3Aofi%2Ffmt%3Akev%3Amtx%3Abook&amp;rft.genre=book&amp;rft.btitle=Machine%20Learning%20with%20Python%20Cookbook&amp;rft.place=Sebastopol&amp;rft.publisher=O%E2%80%B2Reilly&amp;rft.aufirst=Chris&amp;rft.aulast=Albon&amp;rft.au=Chris%20Albon&amp;rft.date=2018-03-23&amp;rft.tpages=366&amp;rft.isbn=978-1-4919-8938-8&amp;rft.language=English"></span>
  <div class="csl-entry" style="margin-bottom: 1em;">Bailer-Jones, C. A. L. (2000) ‘Stellar parameters from very low resolution spectra and medium band filters: Teff, logg and [M/H] using neural networks’, <i>arXiv:astro-ph/0003071</i> [Online]. Available at <a href="http://arxiv.org/abs/astro-ph/0003071">http://arxiv.org/abs/astro-ph/0003071</a> (Accessed 31 May 2018).</div>
  <span class="Z3988" title="url_ver=Z39.88-2004&amp;ctx_ver=Z39.88-2004&amp;rfr_id=info%3Asid%2Fzotero.org%3A2&amp;rft_val_fmt=info%3Aofi%2Ffmt%3Akev%3Amtx%3Ajournal&amp;rft.genre=article&amp;rft.atitle=Stellar%20parameters%20from%20very%20low%20resolution%20spectra%20and%20medium%20band%20filters%3A%20Teff%2C%20logg%20and%20%5BM%2FH%5D%20using%20neural%20networks&amp;rft.jtitle=arXiv%3Aastro-ph%2F0003071&amp;rft.aufirst=Coryn%20A.%20L.&amp;rft.aulast=Bailer-Jones&amp;rft.au=Coryn%20A.%20L.%20Bailer-Jones&amp;rft.date=2000-03-06"></span>
  <div class="csl-entry" style="margin-bottom: 1em;">Bailer-Jones, C. A. L., Irwin, M., Gilmore, G. and von Hippel, T. (1997) ‘Physical parametrization of stellar spectra: the neural network approach’, <i>Monthly Notices of the Royal Astronomical Society</i>, vol. 292, no. 1, pp. 157–166 [Online]. DOI: <a href="https://doi.org/10.1093/mnras/292.1.157">10.1093/mnras/292.1.157</a>.</div>
  <span class="Z3988" title="url_ver=Z39.88-2004&amp;ctx_ver=Z39.88-2004&amp;rfr_id=info%3Asid%2Fzotero.org%3A2&amp;rft_id=info%3Adoi%2F10.1093%2Fmnras%2F292.1.157&amp;rft_val_fmt=info%3Aofi%2Ffmt%3Akev%3Amtx%3Ajournal&amp;rft.genre=article&amp;rft.atitle=Physical%20parametrization%20of%20stellar%20spectra%3A%20the%20neural%20network%20approach&amp;rft.jtitle=Monthly%20Notices%20of%20the%20Royal%20Astronomical%20Society&amp;rft.stitle=Mon%20Not%20R%20Astron%20Soc&amp;rft.volume=292&amp;rft.issue=1&amp;rft.aufirst=Coryn%20A.%20L.&amp;rft.aulast=Bailer-Jones&amp;rft.au=Coryn%20A.%20L.%20Bailer-Jones&amp;rft.au=Mike%20Irwin&amp;rft.au=Gerard%20Gilmore&amp;rft.au=Ted%20von%20Hippel&amp;rft.date=1997-11-21&amp;rft.pages=157-166&amp;rft.spage=157&amp;rft.epage=166&amp;rft.issn=0035-8711&amp;rft.language=en"></span>
  <div class="csl-entry" style="margin-bottom: 1em;">Bakker, I. den (2017) <i>Python Deep Learning Cookbook: Over 75 practical recipes on neural network modeling, reinforcement learning, and transfer learning using Python</i>, Birmingham, Packt Publishing.</div>
  <span class="Z3988" title="url_ver=Z39.88-2004&amp;ctx_ver=Z39.88-2004&amp;rfr_id=info%3Asid%2Fzotero.org%3A2&amp;rft_id=urn%3Aisbn%3A978-1-78712-519-3&amp;rft_val_fmt=info%3Aofi%2Ffmt%3Akev%3Amtx%3Abook&amp;rft.genre=book&amp;rft.btitle=Python%20Deep%20Learning%20Cookbook%3A%20Over%2075%20practical%20recipes%20on%20neural%20network%20modeling%2C%20reinforcement%20learning%2C%20and%20transfer%20learning%20using%20Python&amp;rft.place=Birmingham&amp;rft.publisher=Packt%20Publishing&amp;rft.aufirst=Indra%20den&amp;rft.aulast=Bakker&amp;rft.au=Indra%20den%20Bakker&amp;rft.date=2017-10-27&amp;rft.tpages=330&amp;rft.isbn=978-1-78712-519-3&amp;rft.language=English"></span>
  <div class="csl-entry" style="margin-bottom: 1em;">Bonnin, R. (2016) <i>Building Machine Learning Projects with TensorFlow</i>, Birmingham Mumbai, Packt Publishing.</div>
  <span class="Z3988" title="url_ver=Z39.88-2004&amp;ctx_ver=Z39.88-2004&amp;rfr_id=info%3Asid%2Fzotero.org%3A2&amp;rft_id=urn%3Aisbn%3A978-1-78646-658-7&amp;rft_val_fmt=info%3Aofi%2Ffmt%3Akev%3Amtx%3Abook&amp;rft.genre=book&amp;rft.btitle=Building%20Machine%20Learning%20Projects%20with%20TensorFlow&amp;rft.place=Birmingham%20Mumbai&amp;rft.publisher=Packt%20Publishing&amp;rft.aufirst=Rodolfo&amp;rft.aulast=Bonnin&amp;rft.au=Rodolfo%20Bonnin&amp;rft.date=2016-11-24&amp;rft.tpages=282&amp;rft.isbn=978-1-78646-658-7&amp;rft.language=English"></span>
  <div class="csl-entry" style="margin-bottom: 1em;">Bovy, J. (2018) <i>apogee: Tools for dealing with APOGEE data</i>, Python [Online]. Available at <a href="https://github.com/jobovy/apogee">https://github.com/jobovy/apogee</a> (Accessed 28 May 2018).</div>
  <span class="Z3988" title="url_ver=Z39.88-2004&amp;ctx_ver=Z39.88-2004&amp;rfr_id=info%3Asid%2Fzotero.org%3A2&amp;rft_val_fmt=info%3Aofi%2Ffmt%3Akev%3Amtx%3Adc&amp;rft.type=computerProgram&amp;rft.title=apogee%3A%20Tools%20for%20dealing%20with%20APOGEE%20data&amp;rft.rights=BSD-3-Clause&amp;rft.identifier=https%3A%2F%2Fgithub.com%2Fjobovy%2Fapogee&amp;rft.aufirst=Jo&amp;rft.aulast=Bovy&amp;rft.au=Jo%20Bovy&amp;rft.date=2018-05-10"></span>
  <div class="csl-entry" style="margin-bottom: 1em;">Buduma, N. and Locascio, N. (2017) <i>Fundamentals of Deep Learning: Designing Next-Generation Machine Intelligence Algorithms</i>, 1 edition., Sebastopol, CA, O’Reilly Media.</div>
  <span class="Z3988" title="url_ver=Z39.88-2004&amp;ctx_ver=Z39.88-2004&amp;rfr_id=info%3Asid%2Fzotero.org%3A2&amp;rft_id=urn%3Aisbn%3A978-1-4919-2561-4&amp;rft_val_fmt=info%3Aofi%2Ffmt%3Akev%3Amtx%3Abook&amp;rft.genre=book&amp;rft.btitle=Fundamentals%20of%20Deep%20Learning%3A%20Designing%20Next-Generation%20Machine%20Intelligence%20Algorithms&amp;rft.place=Sebastopol%2C%20CA&amp;rft.publisher=O'Reilly%20Media&amp;rft.edition=1%20edition&amp;rft.aufirst=Nikhil&amp;rft.aulast=Buduma&amp;rft.au=Nikhil%20Buduma&amp;rft.au=Nicholas%20Locascio&amp;rft.date=2017-06-29&amp;rft.tpages=298&amp;rft.isbn=978-1-4919-2561-4&amp;rft.language=English"></span>
  <div class="csl-entry" style="margin-bottom: 1em;">Chollet, F. (2018) <i>Deep Learning with Python</i>, Shelter Island, New York, Manning Publications.</div>
  <span class="Z3988" title="url_ver=Z39.88-2004&amp;ctx_ver=Z39.88-2004&amp;rfr_id=info%3Asid%2Fzotero.org%3A2&amp;rft_id=urn%3Aisbn%3A978-1-61729-443-3&amp;rft_val_fmt=info%3Aofi%2Ffmt%3Akev%3Amtx%3Abook&amp;rft.genre=book&amp;rft.btitle=Deep%20Learning%20with%20Python&amp;rft.place=Shelter%20Island%2C%20New%20York&amp;rft.publisher=Manning%20Publications&amp;rft.aufirst=Francois&amp;rft.aulast=Chollet&amp;rft.au=Francois%20Chollet&amp;rft.date=2018-01-10&amp;rft.tpages=384&amp;rft.isbn=978-1-61729-443-3&amp;rft.language=English"></span>
  <div class="csl-entry" style="margin-bottom: 1em;">Fabbro, S., Venn, K., O’Briain, T., Bialek, S., Kielty, C., Jahandar, F. and Monty, S. (2018) ‘An Application of Deep Neural Networks in the Analysis of Stellar Spectra’, <i>Monthly Notices of the Royal Astronomical Society</i>, vol. 475, no. 3, pp. 2978–2993 [Online]. DOI: <a href="https://doi.org/10.1093/mnras/stx3298">10.1093/mnras/stx3298</a>.</div>
  <span class="Z3988" title="url_ver=Z39.88-2004&amp;ctx_ver=Z39.88-2004&amp;rfr_id=info%3Asid%2Fzotero.org%3A2&amp;rft_id=info%3Adoi%2F10.1093%2Fmnras%2Fstx3298&amp;rft_val_fmt=info%3Aofi%2Ffmt%3Akev%3Amtx%3Ajournal&amp;rft.genre=article&amp;rft.atitle=An%20Application%20of%20Deep%20Neural%20Networks%20in%20the%20Analysis%20of%20Stellar%20Spectra&amp;rft.jtitle=Monthly%20Notices%20of%20the%20Royal%20Astronomical%20Society&amp;rft.volume=475&amp;rft.issue=3&amp;rft.aufirst=Sebastien&amp;rft.aulast=Fabbro&amp;rft.au=Sebastien%20Fabbro&amp;rft.au=Kim%20Venn&amp;rft.au=Teaghan%20O'Briain&amp;rft.au=Spencer%20Bialek&amp;rft.au=Collin%20Kielty&amp;rft.au=Farbod%20Jahandar&amp;rft.au=Stephanie%20Monty&amp;rft.date=2018-04-11&amp;rft.pages=2978-2993&amp;rft.spage=2978&amp;rft.epage=2993&amp;rft.issn=0035-8711%2C%201365-2966"></span>
  <div class="csl-entry" style="margin-bottom: 1em;">Géron, A. (2017) <i>Hands-On Machine Learning with Scikit-Learn and TensorFlow: Concepts, Tools, and Techniques to Build Intelligent Systems</i>, 1 edition., Beijing Boston Farnham Sebastopol Tokyo, O’Reilly Media.</div>
  <span class="Z3988" title="url_ver=Z39.88-2004&amp;ctx_ver=Z39.88-2004&amp;rfr_id=info%3Asid%2Fzotero.org%3A2&amp;rft_id=urn%3Aisbn%3A978-1-4919-6229-9&amp;rft_val_fmt=info%3Aofi%2Ffmt%3Akev%3Amtx%3Abook&amp;rft.genre=book&amp;rft.btitle=Hands-On%20Machine%20Learning%20with%20Scikit-Learn%20and%20TensorFlow%3A%20Concepts%2C%20Tools%2C%20and%20Techniques%20to%20Build%20Intelligent%20Systems&amp;rft.place=Beijing%20Boston%20Farnham%20Sebastopol%20Tokyo&amp;rft.publisher=O'Reilly%20Media&amp;rft.edition=1%20edition&amp;rft.aufirst=Aur%C3%A9lien&amp;rft.aulast=G%C3%A9ron&amp;rft.au=Aur%C3%A9lien%20G%C3%A9ron&amp;rft.date=2017-04-09&amp;rft.tpages=572&amp;rft.isbn=978-1-4919-6229-9&amp;rft.language=English"></span>
  <div class="csl-entry" style="margin-bottom: 1em;">Gray, D. F. (2008) <i>The Observation and Analysis of Stellar Photospheres</i>, 3rd edn.</div>
  <span class="Z3988" title="url_ver=Z39.88-2004&amp;ctx_ver=Z39.88-2004&amp;rfr_id=info%3Asid%2Fzotero.org%3A2&amp;rft_val_fmt=info%3Aofi%2Ffmt%3Akev%3Amtx%3Abook&amp;rft.genre=book&amp;rft.btitle=The%20Observation%20and%20Analysis%20of%20Stellar%20Photospheres&amp;rft.edition=3&amp;rft.aufirst=David%20F.&amp;rft.aulast=Gray&amp;rft.au=David%20F.%20Gray&amp;rft.date=2008"></span>
  <div class="csl-entry" style="margin-bottom: 1em;">Gulli, A. and Pal, S. (2017) <i>Deep Learning with Keras: Implementing deep learning models and neural networks with the power of Python</i>, Birmingham Mumbai, Packt Publishing.</div>
  <span class="Z3988" title="url_ver=Z39.88-2004&amp;ctx_ver=Z39.88-2004&amp;rfr_id=info%3Asid%2Fzotero.org%3A2&amp;rft_id=urn%3Aisbn%3A978-1-78712-842-2&amp;rft_val_fmt=info%3Aofi%2Ffmt%3Akev%3Amtx%3Abook&amp;rft.genre=book&amp;rft.btitle=Deep%20Learning%20with%20Keras%3A%20Implementing%20deep%20learning%20models%20and%20neural%20networks%20with%20the%20power%20of%20Python&amp;rft.place=Birmingham%20Mumbai&amp;rft.publisher=Packt%20Publishing&amp;rft.aufirst=Antonio&amp;rft.aulast=Gulli&amp;rft.au=Antonio%20Gulli&amp;rft.au=Sujit%20Pal&amp;rft.date=2017-04-26&amp;rft.tpages=318&amp;rft.isbn=978-1-78712-842-2&amp;rft.language=English"></span>
  <div class="csl-entry" style="margin-bottom: 1em;">Hafner, D., Davidson, J. and Vanhoucke, V. (2017) ‘TensorFlow Agents: Efficient Batched Reinforcement Learning in TensorFlow’, <i>arXiv:1709.02878 [cs]</i> [Online]. Available at <a href="http://arxiv.org/abs/1709.02878">http://arxiv.org/abs/1709.02878</a> (Accessed 27 May 2018).</div>
  <span class="Z3988" title="url_ver=Z39.88-2004&amp;ctx_ver=Z39.88-2004&amp;rfr_id=info%3Asid%2Fzotero.org%3A2&amp;rft_val_fmt=info%3Aofi%2Ffmt%3Akev%3Amtx%3Ajournal&amp;rft.genre=article&amp;rft.atitle=TensorFlow%20Agents%3A%20Efficient%20Batched%20Reinforcement%20Learning%20in%20TensorFlow&amp;rft.jtitle=arXiv%3A1709.02878%20%5Bcs%5D&amp;rft.aufirst=Danijar&amp;rft.aulast=Hafner&amp;rft.au=Danijar%20Hafner&amp;rft.au=James%20Davidson&amp;rft.au=Vincent%20Vanhoucke&amp;rft.date=2017-09-08"></span>
  <div class="csl-entry" style="margin-bottom: 1em;">Hayden, M. R., Holtzman, J. A., Bovy, J., Majewski, S. R., Johnson, J. A., Prieto, C. A., Beers, T. C., Cunha, K., Frinchaboy, P. M., Pérez, A. E. G., Girardi, L., Hearty, F. R., Lee, Y. S., Nidever, D., Schiavon, R. P., Schlesinger, K. J., Schneider, D. P., Schultheis, M., Shetrone, M., Smith, V. V., Zasowski, G., Bizyaev, D., Diane Feuillet, Hasselquist, S., Kinemuchi, K., Malanushenko, E., Malanushenko, V., Robert O’Connell, Pan, K. and Stassun, K. (2014) ‘Chemical Cartography with APOGEE: Large-scale Mean Metallicity Maps of the Milky Way Disk’, <i>The Astronomical Journal</i>, vol. 147, no. 5, p. 116 [Online]. DOI: <a href="https://doi.org/10.1088/0004-6256/147/5/116">10.1088/0004-6256/147/5/116</a>.</div>
  <span class="Z3988" title="url_ver=Z39.88-2004&amp;ctx_ver=Z39.88-2004&amp;rfr_id=info%3Asid%2Fzotero.org%3A2&amp;rft_id=info%3Adoi%2F10.1088%2F0004-6256%2F147%2F5%2F116&amp;rft_val_fmt=info%3Aofi%2Ffmt%3Akev%3Amtx%3Ajournal&amp;rft.genre=article&amp;rft.atitle=Chemical%20Cartography%20with%20APOGEE%3A%20Large-scale%20Mean%20Metallicity%20Maps%20of%20the%20Milky%20Way%20Disk&amp;rft.jtitle=The%20Astronomical%20Journal&amp;rft.stitle=AJ&amp;rft.volume=147&amp;rft.issue=5&amp;rft.aufirst=Michael%20R.&amp;rft.aulast=Hayden&amp;rft.au=Michael%20R.%20Hayden&amp;rft.au=Jon%20A.%20Holtzman&amp;rft.au=Jo%20Bovy&amp;rft.au=Steven%20R.%20Majewski&amp;rft.au=Jennifer%20A.%20Johnson&amp;rft.au=Carlos%20Allende%20Prieto&amp;rft.au=Timothy%20C.%20Beers&amp;rft.au=Katia%20Cunha&amp;rft.au=Peter%20M.%20Frinchaboy&amp;rft.au=Ana%20E.%20Garc%C3%ADa%20P%C3%A9rez&amp;rft.au=L%C3%A9o%20Girardi&amp;rft.au=Fred%20R.%20Hearty&amp;rft.au=Young%20Sun%20Lee&amp;rft.au=David%20Nidever&amp;rft.au=Ricardo%20P.%20Schiavon&amp;rft.au=Katharine%20J.%20Schlesinger&amp;rft.au=Donald%20P.%20Schneider&amp;rft.au=Mathias%20Schultheis&amp;rft.au=Matthew%20Shetrone&amp;rft.au=Verne%20V.%20Smith&amp;rft.au=Gail%20Zasowski&amp;rft.au=Dmitry%20Bizyaev&amp;rft.au=undefined&amp;rft.au=Sten%20Hasselquist&amp;rft.au=Karen%20Kinemuchi&amp;rft.au=Elena%20Malanushenko&amp;rft.au=Viktor%20Malanushenko&amp;rft.au=undefined&amp;rft.au=Kaike%20Pan&amp;rft.au=Keivan%20Stassun&amp;rft.date=2014&amp;rft.pages=116&amp;rft.issn=1538-3881&amp;rft.language=en"></span>
  <div class="csl-entry" style="margin-bottom: 1em;">von Hippel, T., Storrie-Lombardi, L. J., Storrie-Lombardi, M. C. and Irwin, M. J. (1994) ‘Automated classification of stellar spectra – I. Initial results with artificial neural networks’, <i>Monthly Notices of the Royal Astronomical Society</i>, vol. 269, no. 1, pp. 97–104 [Online]. DOI: <a href="https://doi.org/10.1093/mnras/269.1.97">10.1093/mnras/269.1.97</a>.</div>
  <span class="Z3988" title="url_ver=Z39.88-2004&amp;ctx_ver=Z39.88-2004&amp;rfr_id=info%3Asid%2Fzotero.org%3A2&amp;rft_id=info%3Adoi%2F10.1093%2Fmnras%2F269.1.97&amp;rft_val_fmt=info%3Aofi%2Ffmt%3Akev%3Amtx%3Ajournal&amp;rft.genre=article&amp;rft.atitle=Automated%20classification%20of%20stellar%20spectra%20%E2%80%93%20I.%20Initial%20results%20with%20artificial%20neural%20networks&amp;rft.jtitle=Monthly%20Notices%20of%20the%20Royal%20Astronomical%20Society&amp;rft.stitle=Mon%20Not%20R%20Astron%20Soc&amp;rft.volume=269&amp;rft.issue=1&amp;rft.aufirst=T.&amp;rft.aulast=von%20Hippel&amp;rft.au=T.%20von%20Hippel&amp;rft.au=L.%20J.%20Storrie-Lombardi&amp;rft.au=M.%20C.%20Storrie-Lombardi&amp;rft.au=M.%20J.%20Irwin&amp;rft.date=1994-07-01&amp;rft.pages=97-104&amp;rft.spage=97&amp;rft.epage=104&amp;rft.issn=0035-8711&amp;rft.language=en"></span>
  <div class="csl-entry" style="margin-bottom: 1em;">Holtzman, J., Nidever, D., Garcia-Perez, A. and Allende-Prieto, C. (n.d.) <i>STATUS OF APOGEE DATA PRODUCTS, REDUCTION, AND ANALYSIS. - ppt download</i> [Online]. Available at <a href="http://slideplayer.com/slide/9047384/#">http://slideplayer.com/slide/9047384/#</a> (Accessed 28 May 2018).</div>
  <span class="Z3988" title="url_ver=Z39.88-2004&amp;ctx_ver=Z39.88-2004&amp;rfr_id=info%3Asid%2Fzotero.org%3A2&amp;rft_val_fmt=info%3Aofi%2Ffmt%3Akev%3Amtx%3Adc&amp;rft.type=webpage&amp;rft.title=STATUS%20OF%20APOGEE%20DATA%20PRODUCTS%2C%20REDUCTION%2C%20AND%20ANALYSIS.%20-%20ppt%20download&amp;rft.identifier=http%3A%2F%2Fslideplayer.com%2Fslide%2F9047384%2F%23&amp;rft.aufirst=Jon&amp;rft.aulast=Holtzman&amp;rft.au=Jon%20Holtzman&amp;rft.au=David%20Nidever&amp;rft.au=Ana%20Garcia-Perez&amp;rft.au=Carlos%20Allende-Prieto"></span>
  <div class="csl-entry" style="margin-bottom: 1em;">Hope, T., Resheff, Y. S. and Lieder, I. (2017) <i>Learning TensorFlow: A Guide to Building Deep Learning Systems</i>, 1 edition., Beijing Boston Farnham Sebastopol Tokyo, O’Reilly Media.</div>
  <span class="Z3988" title="url_ver=Z39.88-2004&amp;ctx_ver=Z39.88-2004&amp;rfr_id=info%3Asid%2Fzotero.org%3A2&amp;rft_id=urn%3Aisbn%3A978-1-4919-7851-1&amp;rft_val_fmt=info%3Aofi%2Ffmt%3Akev%3Amtx%3Abook&amp;rft.genre=book&amp;rft.btitle=Learning%20TensorFlow%3A%20A%20Guide%20to%20Building%20Deep%20Learning%20Systems&amp;rft.place=Beijing%20Boston%20Farnham%20Sebastopol%20Tokyo&amp;rft.publisher=O'Reilly%20Media&amp;rft.edition=1%20edition&amp;rft.aufirst=Tom&amp;rft.aulast=Hope&amp;rft.au=Tom%20Hope&amp;rft.au=Yehezkel%20S.%20Resheff&amp;rft.au=Itay%20Lieder&amp;rft.date=2017-08-27&amp;rft.tpages=242&amp;rft.isbn=978-1-4919-7851-1&amp;rft.language=English"></span>
  <div class="csl-entry" style="margin-bottom: 1em;"><a href="http://astronn.readthedocs.io/en/latest/">http://astronn.readthedocs.io/en/latest/</a> (n.d.) <i>Welcome to astroNN’s documentation! — astroNN 0.9.2.9 Alpha documentation</i> [Online]. (Accessed 15 June 2018).</div>
  <span class="Z3988" title="url_ver=Z39.88-2004&amp;ctx_ver=Z39.88-2004&amp;rfr_id=info%3Asid%2Fzotero.org%3A2&amp;rft_val_fmt=info%3Aofi%2Ffmt%3Akev%3Amtx%3Adc&amp;rft.type=webpage&amp;rft.title=Welcome%20to%20astroNN%E2%80%99s%20documentation!%20%E2%80%94%20astroNN%200.9.2.9%20Alpha%20documentation&amp;rft.identifier=http%3A%2F%2Fastronn.readthedocs.io%2Fen%2Flatest%2F"></span>
  <div class="csl-entry" style="margin-bottom: 1em;">Ketkar, N. (2017) <i>Deep Learning with Python: A Hands-on Introduction</i>, 1st ed. edition., New York, NY, Apress.</div>
  <span class="Z3988" title="url_ver=Z39.88-2004&amp;ctx_ver=Z39.88-2004&amp;rfr_id=info%3Asid%2Fzotero.org%3A2&amp;rft_id=urn%3Aisbn%3A978-1-4842-2765-7&amp;rft_val_fmt=info%3Aofi%2Ffmt%3Akev%3Amtx%3Abook&amp;rft.genre=book&amp;rft.btitle=Deep%20Learning%20with%20Python%3A%20A%20Hands-on%20Introduction&amp;rft.place=New%20York%2C%20NY&amp;rft.publisher=Apress&amp;rft.edition=1st%20ed.%20edition&amp;rft.aufirst=Nikhil&amp;rft.aulast=Ketkar&amp;rft.au=Nikhil%20Ketkar&amp;rft.date=2017-04-19&amp;rft.tpages=226&amp;rft.isbn=978-1-4842-2765-7&amp;rft.language=English"></span>
  <div class="csl-entry" style="margin-bottom: 1em;">Kielty, C. L. (2017) ‘New frontiers in galactic archaeology: spectroscopic surveys, carbon-enhanced metal-poor stars, and machine learning applications’, Thesis [Online]. Available at <a href="https://dspace.library.uvic.ca//handle/1828/8646">https://dspace.library.uvic.ca//handle/1828/8646</a> (Accessed 31 May 2018).</div>
  <span class="Z3988" title="url_ver=Z39.88-2004&amp;ctx_ver=Z39.88-2004&amp;rfr_id=info%3Asid%2Fzotero.org%3A2&amp;rft_val_fmt=info%3Aofi%2Ffmt%3Akev%3Amtx%3Adissertation&amp;rft.title=New%20frontiers%20in%20galactic%20archaeology%3A%20spectroscopic%20surveys%2C%20carbon-enhanced%20metal-poor%20stars%2C%20and%20machine%20learning%20applications&amp;rft.aufirst=Collin%20Louis&amp;rft.aulast=Kielty&amp;rft.au=Collin%20Louis%20Kielty&amp;rft.date=2017&amp;rft.language=en"></span>
  <div class="csl-entry" style="margin-bottom: 1em;">Li, X.-R., Pan, R.-Y. and Duan, F.-Q. (2017) ‘Parameterizing Stellar Spectra Using Deep Neural Networks’, <i>Research in Astronomy and Astrophysics</i>, vol. 17, no. 4, p. 036 [Online]. DOI: <a href="https://doi.org/10.1088/1674-4527/17/4/36">10.1088/1674-4527/17/4/36</a>.</div>
  <span class="Z3988" title="url_ver=Z39.88-2004&amp;ctx_ver=Z39.88-2004&amp;rfr_id=info%3Asid%2Fzotero.org%3A2&amp;rft_id=info%3Adoi%2F10.1088%2F1674-4527%2F17%2F4%2F36&amp;rft_val_fmt=info%3Aofi%2Ffmt%3Akev%3Amtx%3Ajournal&amp;rft.genre=article&amp;rft.atitle=Parameterizing%20Stellar%20Spectra%20Using%20Deep%20Neural%20Networks&amp;rft.jtitle=Research%20in%20Astronomy%20and%20Astrophysics&amp;rft.stitle=Res.%20Astron.%20Astrophys.&amp;rft.volume=17&amp;rft.issue=4&amp;rft.aufirst=Xiang-Ru&amp;rft.aulast=Li&amp;rft.au=Xiang-Ru%20Li&amp;rft.au=Ru-Yang%20Pan&amp;rft.au=Fu-Qing%20Duan&amp;rft.date=2017&amp;rft.pages=036&amp;rft.issn=1674-4527&amp;rft.language=en"></span>
  <div class="csl-entry" style="margin-bottom: 1em;">Majewski, S. R., Schiavon, R. P., Frinchaboy, P. M., Prieto, C. A., Robert Barkhouser, Bizyaev, D., Blank, B., Brunner, S., Burton, A., Carrera, R., Chojnowski, S. D., Cunha, K., Epstein, C., Fitzgerald, G., Pérez, A. E. G., Hearty, F. R., Chuck Henderson, Holtzman, J. A., Johnson, J. A., Lam, C. R., Lawler, J. E., Maseman, P., Szabolcs Mészáros, Nelson, M., Nguyen, D. C., Nidever, D. L., Pinsonneault, M., Shetrone, M., Stephen Smee, Smith, V. V., Stolberg, T., Skrutskie, M. F., Walker, E., Wilson, J. C., Gail Zasowski, Anders, F., Basu, S., Beland, S., Blanton, M. R., Bovy, J., Brownstein, J. R., Carlberg, J., Chaplin, W., Chiappini, C., Eisenstein, D. J., Yvonne Elsworth, Feuillet, D., Fleming, S. W., Galbraith-Frew, J., García, R. A., García-Hernández, D. A., Gillespie, B. A., Girardi, L., Gunn, J. E., Hasselquist, S., Hayden, M. R., Hekker, S., Ivans, I., Kinemuchi, K., Klaene, M., Mahadevan, S., Mathur, S., Benoît Mosser, Muna, D., Munn, J. A., Nichol, R. C., O’Connell, R. W., Parejko, J. K., Robin, A. C., Rocha-Pinto, H., Schultheis, M., Serenelli, A. M., Shane, N., Aguirre, V. S., Sobeck, J. S., Thompson, B., Troup, N. W., Weinberg, D. H. and Zamora, O. (2017) ‘The Apache Point Observatory Galactic Evolution Experiment (APOGEE)’, <i>The Astronomical Journal</i>, vol. 154, no. 3, p. 94 [Online]. DOI: <a href="https://doi.org/10.3847/1538-3881/aa784d">10.3847/1538-3881/aa784d</a>.</div>
  <span class="Z3988" title="url_ver=Z39.88-2004&amp;ctx_ver=Z39.88-2004&amp;rfr_id=info%3Asid%2Fzotero.org%3A2&amp;rft_id=info%3Adoi%2F10.3847%2F1538-3881%2Faa784d&amp;rft_val_fmt=info%3Aofi%2Ffmt%3Akev%3Amtx%3Ajournal&amp;rft.genre=article&amp;rft.atitle=The%20Apache%20Point%20Observatory%20Galactic%20Evolution%20Experiment%20(APOGEE)&amp;rft.jtitle=The%20Astronomical%20Journal&amp;rft.stitle=AJ&amp;rft.volume=154&amp;rft.issue=3&amp;rft.aufirst=Steven%20R.&amp;rft.aulast=Majewski&amp;rft.au=Steven%20R.%20Majewski&amp;rft.au=Ricardo%20P.%20Schiavon&amp;rft.au=Peter%20M.%20Frinchaboy&amp;rft.au=Carlos%20Allende%20Prieto&amp;rft.au=undefined&amp;rft.au=Dmitry%20Bizyaev&amp;rft.au=Basil%20Blank&amp;rft.au=Sophia%20Brunner&amp;rft.au=Adam%20Burton&amp;rft.au=Ricardo%20Carrera&amp;rft.au=S.%20Drew%20Chojnowski&amp;rft.au=K%C3%A1tia%20Cunha&amp;rft.au=Courtney%20Epstein&amp;rft.au=Greg%20Fitzgerald&amp;rft.au=Ana%20E.%20Garc%C3%ADa%20P%C3%A9rez&amp;rft.au=Fred%20R.%20Hearty&amp;rft.au=undefined&amp;rft.au=Jon%20A.%20Holtzman&amp;rft.au=Jennifer%20A.%20Johnson&amp;rft.au=Charles%20R.%20Lam&amp;rft.au=James%20E.%20Lawler&amp;rft.au=Paul%20Maseman&amp;rft.au=undefined&amp;rft.au=Matthew%20Nelson&amp;rft.au=Duy%20Coung%20Nguyen&amp;rft.au=David%20L.%20Nidever&amp;rft.au=Marc%20Pinsonneault&amp;rft.au=Matthew%20Shetrone&amp;rft.au=undefined&amp;rft.au=Verne%20V.%20Smith&amp;rft.au=Todd%20Stolberg&amp;rft.au=Michael%20F.%20Skrutskie&amp;rft.au=Eric%20Walker&amp;rft.au=John%20C.%20Wilson&amp;rft.au=undefined&amp;rft.au=Friedrich%20Anders&amp;rft.au=Sarbani%20Basu&amp;rft.au=Stephane%20Beland&amp;rft.au=Michael%20R.%20Blanton&amp;rft.au=Jo%20Bovy&amp;rft.au=Joel%20R.%20Brownstein&amp;rft.au=Joleen%20Carlberg&amp;rft.au=William%20Chaplin&amp;rft.au=Cristina%20Chiappini&amp;rft.au=Daniel%20J.%20Eisenstein&amp;rft.au=undefined&amp;rft.au=Diane%20Feuillet&amp;rft.au=Scott%20W.%20Fleming&amp;rft.au=Jessica%20Galbraith-Frew&amp;rft.au=Rafael%20A.%20Garc%C3%ADa&amp;rft.au=D.%20An%C3%ADbal%20Garc%C3%ADa-Hern%C3%A1ndez&amp;rft.au=Bruce%20A.%20Gillespie&amp;rft.au=L%C3%A9o%20Girardi&amp;rft.au=James%20E.%20Gunn&amp;rft.au=Sten%20Hasselquist&amp;rft.au=Michael%20R.%20Hayden&amp;rft.au=Saskia%20Hekker&amp;rft.au=Inese%20Ivans&amp;rft.au=Karen%20Kinemuchi&amp;rft.au=Mark%20Klaene&amp;rft.au=Suvrath%20Mahadevan&amp;rft.au=Savita%20Mathur&amp;rft.au=undefined&amp;rft.au=Demitri%20Muna&amp;rft.au=Jeffrey%20A.%20Munn&amp;rft.au=Robert%20C.%20Nichol&amp;rft.au=Robert%20W.%20O%E2%80%99Connell&amp;rft.au=John%20K.%20Parejko&amp;rft.au=A.%20C.%20Robin&amp;rft.au=Helio%20Rocha-Pinto&amp;rft.au=Matthias%20Schultheis&amp;rft.au=Aldo%20M.%20Serenelli&amp;rft.au=Neville%20Shane&amp;rft.au=Victor%20Silva%20Aguirre&amp;rft.au=Jennifer%20S.%20Sobeck&amp;rft.au=Benjamin%20Thompson&amp;rft.au=Nicholas%20W.%20Troup&amp;rft.au=David%20H.%20Weinberg&amp;rft.au=Olga%20Zamora&amp;rft.date=2017&amp;rft.pages=94&amp;rft.issn=1538-3881&amp;rft.language=en"></span>
  <div class="csl-entry" style="margin-bottom: 1em;">Manteiga, M., Ordóñez, D., Dafonte, C. and Arcay, B. (2010) ‘ANNs and Wavelets: A Strategy for Gaia RVS Low S/N Stellar Spectra Parameterization’, <i>Publications of the Astronomical Society of the Pacific</i>, vol. 122, no. 891, p. 608 [Online]. DOI: <a href="https://doi.org/10.1086/653039">10.1086/653039</a>.</div>
  <span class="Z3988" title="url_ver=Z39.88-2004&amp;ctx_ver=Z39.88-2004&amp;rfr_id=info%3Asid%2Fzotero.org%3A2&amp;rft_id=info%3Adoi%2F10.1086%2F653039&amp;rft_val_fmt=info%3Aofi%2Ffmt%3Akev%3Amtx%3Ajournal&amp;rft.genre=article&amp;rft.atitle=ANNs%20and%20Wavelets%3A%20A%20Strategy%20for%20Gaia%20RVS%20Low%20S%2FN%20Stellar%20Spectra%20Parameterization&amp;rft.jtitle=Publications%20of%20the%20Astronomical%20Society%20of%20the%20Pacific&amp;rft.volume=122&amp;rft.issue=891&amp;rft.aufirst=M.&amp;rft.aulast=Manteiga&amp;rft.au=M.%20Manteiga&amp;rft.au=D.%20Ord%C3%B3%C3%B1ez&amp;rft.au=C.%20Dafonte&amp;rft.au=B.%20Arcay&amp;rft.date=2010-05&amp;rft.pages=608&amp;rft.language=en"></span>
  <div class="csl-entry" style="margin-bottom: 1em;">Martelli, A., Ravenscroft, A. and Holden, S. (2017) <i>Python in a Nutshell: A Desktop Quick Reference</i>, 3rd Revised edition edition., Sebastopol, CA, O′Reilly.</div>
  <span class="Z3988" title="url_ver=Z39.88-2004&amp;ctx_ver=Z39.88-2004&amp;rfr_id=info%3Asid%2Fzotero.org%3A2&amp;rft_id=urn%3Aisbn%3A978-1-4493-9292-5&amp;rft_val_fmt=info%3Aofi%2Ffmt%3Akev%3Amtx%3Abook&amp;rft.genre=book&amp;rft.btitle=Python%20in%20a%20Nutshell%3A%20A%20Desktop%20Quick%20Reference&amp;rft.place=Sebastopol%2C%20CA&amp;rft.publisher=O%E2%80%B2Reilly&amp;rft.edition=3rd%20Revised%20edition%20edition&amp;rft.aufirst=Alex&amp;rft.aulast=Martelli&amp;rft.au=Alex%20Martelli&amp;rft.au=Anna%20Ravenscroft&amp;rft.au=Steve%20Holden&amp;rft.date=2017-04-25&amp;rft.tpages=772&amp;rft.isbn=978-1-4493-9292-5&amp;rft.language=English"></span>
  <div class="csl-entry" style="margin-bottom: 1em;">Mukhopadhyay, S. (2018) <i>Advanced Data Analytics Using Python: With Machine Learning, Deep Learning and NLP Examples</i>, 1st ed. edition., New York, NY, Apress.</div>
  <span class="Z3988" title="url_ver=Z39.88-2004&amp;ctx_ver=Z39.88-2004&amp;rfr_id=info%3Asid%2Fzotero.org%3A2&amp;rft_id=urn%3Aisbn%3A978-1-4842-3449-5&amp;rft_val_fmt=info%3Aofi%2Ffmt%3Akev%3Amtx%3Abook&amp;rft.genre=book&amp;rft.btitle=Advanced%20Data%20Analytics%20Using%20Python%3A%20With%20Machine%20Learning%2C%20Deep%20Learning%20and%20NLP%20Examples&amp;rft.place=New%20York%2C%20NY&amp;rft.publisher=Apress&amp;rft.edition=1st%20ed.%20edition&amp;rft.aufirst=Sayan&amp;rft.aulast=Mukhopadhyay&amp;rft.au=Sayan%20Mukhopadhyay&amp;rft.date=2018-03-29&amp;rft.tpages=186&amp;rft.isbn=978-1-4842-3449-5&amp;rft.language=English"></span>
  <div class="csl-entry" style="margin-bottom: 1em;">Nandy, A. and Biswas, M. (2017) <i>Reinforcement Learning: With Open AI, TensorFlow and Keras Using Python</i>, 1st ed. edition., New York, NY, Apress.</div>
  <span class="Z3988" title="url_ver=Z39.88-2004&amp;ctx_ver=Z39.88-2004&amp;rfr_id=info%3Asid%2Fzotero.org%3A2&amp;rft_id=urn%3Aisbn%3A978-1-4842-3284-2&amp;rft_val_fmt=info%3Aofi%2Ffmt%3Akev%3Amtx%3Abook&amp;rft.genre=book&amp;rft.btitle=Reinforcement%20Learning%3A%20With%20Open%20AI%2C%20TensorFlow%20and%20Keras%20Using%20Python&amp;rft.place=New%20York%2C%20NY&amp;rft.publisher=Apress&amp;rft.edition=1st%20ed.%20edition&amp;rft.aufirst=Abhishek&amp;rft.aulast=Nandy&amp;rft.au=Abhishek%20Nandy&amp;rft.au=Manisha%20Biswas&amp;rft.date=2017-12-08&amp;rft.tpages=167&amp;rft.isbn=978-1-4842-3284-2&amp;rft.language=English"></span>
  <div class="csl-entry" style="margin-bottom: 1em;">O’Briain, T. and Fabbro, S. (2017) <i>starnet: Stellar spectra features with machine learning</i>, Jupyter Notebook, astroai [Online]. Available at <a href="https://github.com/astroai/starnet">https://github.com/astroai/starnet</a> (Accessed 28 May 2018).</div>
  <span class="Z3988" title="url_ver=Z39.88-2004&amp;ctx_ver=Z39.88-2004&amp;rfr_id=info%3Asid%2Fzotero.org%3A2&amp;rft_val_fmt=info%3Aofi%2Ffmt%3Akev%3Amtx%3Adc&amp;rft.type=computerProgram&amp;rft.title=starnet%3A%20Stellar%20spectra%20features%20with%20machine%20learning&amp;rft.rights=BSD-2-Clause&amp;rft.identifier=https%3A%2F%2Fgithub.com%2Fastroai%2Fstarnet&amp;rft.aufirst=Teaghan&amp;rft.aulast=O'Briain&amp;rft.au=Teaghan%20O'Briain&amp;rft.au=Sebastien%20Fabbro&amp;rft.date=2017-11-02"></span>
  <div class="csl-entry" style="margin-bottom: 1em;">Pattanayak, S. (2017) <i>Pro Deep Learning with TensorFlow: A Mathematical Approach to Advanced Artificial Intelligence in Python</i>, 1st ed. edition., New York, NY, Apress.</div>
  <span class="Z3988" title="url_ver=Z39.88-2004&amp;ctx_ver=Z39.88-2004&amp;rfr_id=info%3Asid%2Fzotero.org%3A2&amp;rft_id=urn%3Aisbn%3A978-1-4842-3095-4&amp;rft_val_fmt=info%3Aofi%2Ffmt%3Akev%3Amtx%3Abook&amp;rft.genre=book&amp;rft.btitle=Pro%20Deep%20Learning%20with%20TensorFlow%3A%20A%20Mathematical%20Approach%20to%20Advanced%20Artificial%20Intelligence%20in%20Python&amp;rft.place=New%20York%2C%20NY&amp;rft.publisher=Apress&amp;rft.edition=1st%20ed.%20edition&amp;rft.aufirst=Santanu&amp;rft.aulast=Pattanayak&amp;rft.au=Santanu%20Pattanayak&amp;rft.date=2017-12-07&amp;rft.tpages=398&amp;rft.isbn=978-1-4842-3095-4&amp;rft.language=English"></span>
  <div class="csl-entry" style="margin-bottom: 1em;">García PérezPérez, A. E., Prieto, C. A., Holtzman, J. A., Shetrone, M., Mészáros, S., Dmitry Bizyaev, Carrera, R., Cunha, K., García-Hernández, D. A., Johnson, J. A., Majewski, S. R., Nidever, D. L., Schiavon, R. P., Shane, N., Smith, V. V., Sobeck, J., Nicholas Troup, Zamora, O., Weinberg, D. H., Bovy, J., Eisenstein, D. J., Feuillet, D., Frinchaboy, P. M., Hayden, M. R., Hearty, F. R., Nguyen, D. C., O’Connell, R. W., Pinsonneault, M. H., Wilson, J. C. and Zasowski, G. (2016) ‘ASPCAP: The APOGEE Stellar Parameter and Chemical Abundances Pipeline’, <i>The Astronomical Journal</i>, vol. 151, no. 6, p. 144 [Online]. DOI: <a href="https://doi.org/10.3847/0004-6256/151/6/144">10.3847/0004-6256/151/6/144</a>.</div>
  <span class="Z3988" title="url_ver=Z39.88-2004&amp;ctx_ver=Z39.88-2004&amp;rfr_id=info%3Asid%2Fzotero.org%3A2&amp;rft_id=info%3Adoi%2F10.3847%2F0004-6256%2F151%2F6%2F144&amp;rft_val_fmt=info%3Aofi%2Ffmt%3Akev%3Amtx%3Ajournal&amp;rft.genre=article&amp;rft.atitle=ASPCAP%3A%20The%20APOGEE%20Stellar%20Parameter%20and%20Chemical%20Abundances%20Pipeline&amp;rft.jtitle=The%20Astronomical%20Journal&amp;rft.stitle=AJ&amp;rft.volume=151&amp;rft.issue=6&amp;rft.aufirst=Ana%20E.%20Garc%C3%ADa&amp;rft.aulast=P%C3%A9rez&amp;rft.au=Ana%20E.%20Garc%C3%ADa%20P%C3%A9rez&amp;rft.au=Carlos%20Allende%20Prieto&amp;rft.au=Jon%20A.%20Holtzman&amp;rft.au=Matthew%20Shetrone&amp;rft.au=Szabolcs%20M%C3%A9sz%C3%A1ros&amp;rft.au=undefined&amp;rft.au=Ricardo%20Carrera&amp;rft.au=Katia%20Cunha&amp;rft.au=D.%20A.%20Garc%C3%ADa-Hern%C3%A1ndez&amp;rft.au=Jennifer%20A.%20Johnson&amp;rft.au=Steven%20R.%20Majewski&amp;rft.au=David%20L.%20Nidever&amp;rft.au=Ricardo%20P.%20Schiavon&amp;rft.au=Neville%20Shane&amp;rft.au=Verne%20V.%20Smith&amp;rft.au=Jennifer%20Sobeck&amp;rft.au=undefined&amp;rft.au=Olga%20Zamora&amp;rft.au=David%20H.%20Weinberg&amp;rft.au=Jo%20Bovy&amp;rft.au=Daniel%20J.%20Eisenstein&amp;rft.au=Diane%20Feuillet&amp;rft.au=Peter%20M.%20Frinchaboy&amp;rft.au=Michael%20R.%20Hayden&amp;rft.au=Fred%20R.%20Hearty&amp;rft.au=Duy%20C.%20Nguyen&amp;rft.au=Robert%20W.%20O%E2%80%99Connell&amp;rft.au=Marc%20H.%20Pinsonneault&amp;rft.au=John%20C.%20Wilson&amp;rft.au=Gail%20Zasowski&amp;rft.date=2016&amp;rft.pages=144&amp;rft.issn=1538-3881&amp;rft.language=en"></span>
  <div class="csl-entry" style="margin-bottom: 1em;">Pinsonneault, M. H., Elsworth, Y., Epstein, C., Hekker, S., Mészáros, S., Chaplin, W. J., Johnson, J. A., García, R. A., Holtzman, J., Mathur, S., Pérez, A. G., Aguirre, V. S., Girardi, L., Basu, S., Shetrone, M., Stello, D., Prieto, C. A., An, D., Beck, P., Beers, T. C., Bizyaev, D., Bloemen, S., Bovy, J., Cunha, K., De Ridder, J., Frinchaboy, P. M., Garcia-Hernández, D. A., Gilliland, R., Harding, P., Hearty, F. R., Huber, D., Ivans, I., Kallinger, T., Majewski, S. R., Metcalfe, T. S., Miglio, A., Mosser, B., Muna, D., Nidever, D. L., Schneider, D. P., Serenelli, A., Smith, V. V., Tayar, J., Zamora, O. and Zasowski, G. (2014) ‘The APOKASC Catalog: An Asteroseismic and Spectroscopic Joint Survey of Targets in the Kepler Fields’, <i>The Astrophysical Journal Supplement Series</i>, vol. 215, no. 2, p. 19 [Online]. DOI: <a href="https://doi.org/10.1088/0067-0049/215/2/19">10.1088/0067-0049/215/2/19</a>.</div>
  <span class="Z3988" title="url_ver=Z39.88-2004&amp;ctx_ver=Z39.88-2004&amp;rfr_id=info%3Asid%2Fzotero.org%3A2&amp;rft_id=info%3Adoi%2F10.1088%2F0067-0049%2F215%2F2%2F19&amp;rft_val_fmt=info%3Aofi%2Ffmt%3Akev%3Amtx%3Ajournal&amp;rft.genre=article&amp;rft.atitle=The%20APOKASC%20Catalog%3A%20An%20Asteroseismic%20and%20Spectroscopic%20Joint%20Survey%20of%20Targets%20in%20the%20Kepler%20Fields&amp;rft.jtitle=The%20Astrophysical%20Journal%20Supplement%20Series&amp;rft.volume=215&amp;rft.issue=2&amp;rft.aufirst=Marc%20H.&amp;rft.aulast=Pinsonneault&amp;rft.au=Marc%20H.%20Pinsonneault&amp;rft.au=Yvonne%20Elsworth&amp;rft.au=Courtney%20Epstein&amp;rft.au=Saskia%20Hekker&amp;rft.au=Sz%20M%C3%A9sz%C3%A1ros&amp;rft.au=William%20J.%20Chaplin&amp;rft.au=Jennifer%20A.%20Johnson&amp;rft.au=Rafael%20A.%20Garc%C3%ADa&amp;rft.au=Jon%20Holtzman&amp;rft.au=Savita%20Mathur&amp;rft.au=Ana%20Garc%C3%ADa%20P%C3%A9rez&amp;rft.au=Victor%20Silva%20Aguirre&amp;rft.au=L%C3%A9o%20Girardi&amp;rft.au=Sarbani%20Basu&amp;rft.au=Matthew%20Shetrone&amp;rft.au=Dennis%20Stello&amp;rft.au=Carlos%20Allende%20Prieto&amp;rft.au=Deokkeun%20An&amp;rft.au=Paul%20Beck&amp;rft.au=Timothy%20C.%20Beers&amp;rft.au=Dmitry%20Bizyaev&amp;rft.au=Steven%20Bloemen&amp;rft.au=Jo%20Bovy&amp;rft.au=Katia%20Cunha&amp;rft.au=Joris%20De%20Ridder&amp;rft.au=Peter%20M.%20Frinchaboy&amp;rft.au=D.%20A.%20Garcia-Hern%C3%A1ndez&amp;rft.au=Ronald%20Gilliland&amp;rft.au=Paul%20Harding&amp;rft.au=Fred%20R.%20Hearty&amp;rft.au=Daniel%20Huber&amp;rft.au=Inese%20Ivans&amp;rft.au=Thomas%20Kallinger&amp;rft.au=Steven%20R.%20Majewski&amp;rft.au=Travis%20S.%20Metcalfe&amp;rft.au=Andrea%20Miglio&amp;rft.au=Benoit%20Mosser&amp;rft.au=Demitri%20Muna&amp;rft.au=David%20L.%20Nidever&amp;rft.au=Donald%20P.%20Schneider&amp;rft.au=Aldo%20Serenelli&amp;rft.au=Verne%20V.%20Smith&amp;rft.au=Jamie%20Tayar&amp;rft.au=Olga%20Zamora&amp;rft.au=Gail%20Zasowski&amp;rft.date=2014-12-02&amp;rft.pages=19&amp;rft.issn=1538-4365"></span>
  <div class="csl-entry" style="margin-bottom: 1em;">Rossant, C. (2018) <i>IPython Interactive Computing and Visualization Cookbook - Second Edition: Over 100 hands-on recipes to sharpen your skills in high-performance ... and data science in the Jupyter Notebook</i>, 2nd Revised edition edition., Packt Publishing.</div>
  <span class="Z3988" title="url_ver=Z39.88-2004&amp;ctx_ver=Z39.88-2004&amp;rfr_id=info%3Asid%2Fzotero.org%3A2&amp;rft_id=urn%3Aisbn%3A978-1-78588-863-2&amp;rft_val_fmt=info%3Aofi%2Ffmt%3Akev%3Amtx%3Abook&amp;rft.genre=book&amp;rft.btitle=IPython%20Interactive%20Computing%20and%20Visualization%20Cookbook%20-%20Second%20Edition%3A%20Over%20100%20hands-on%20recipes%20to%20sharpen%20your%20skills%20in%20high-performance%20...%20and%20data%20science%20in%20the%20Jupyter%20Notebook&amp;rft.publisher=Packt%20Publishing&amp;rft.edition=2nd%20Revised%20edition%20edition&amp;rft.aufirst=Cyrille&amp;rft.aulast=Rossant&amp;rft.au=Cyrille%20Rossant&amp;rft.date=2018-01-31&amp;rft.tpages=548&amp;rft.isbn=978-1-78588-863-2&amp;rft.language=English"></span>
  <div class="csl-entry" style="margin-bottom: 1em;">Schiavon, R. P., Zamora, O., Carrera, R., Lucatello, S., Robin, A. C., Ness, M., Martell, S. L., Smith, V. V., García-Hernández, D. A., Manchado, A., Schönrich, R., Bastian, N., Chiappini, C., Shetrone, M., Mackereth, J. T., Williams, R. A., Mészáros, S., Allende Prieto, C., Anders, F., Bizyaev, D., Beers, T. C., Chojnowski, S. D., Cunha, K., Epstein, C., Frinchaboy, P. M., Pérez, G., E, A., Hearty, F. R., Holtzman, J. A., Johnson, J. A., Kinemuchi, K., Majewski, S. R., Muna, D., Nidever, D. L., Nguyen, D. C., O’Connell, R. W., Oravetz, D., Pan, K., Pinsonneault, M., Schneider, D. P., Schultheis, M., Simmons, A., Skrutskie, M. F., Sobeck, J., Wilson, J. C. and Zasowski, G. (2017) ‘Chemical tagging with APOGEE: discovery of a large population of N-rich stars in the inner Galaxy’, <i>Monthly Notices of the Royal Astronomical Society</i>, vol. 465, no. 1, pp. 501–524 [Online]. DOI: <a href="https://doi.org/10.1093/mnras/stw2162">10.1093/mnras/stw2162</a>.</div>
  <span class="Z3988" title="url_ver=Z39.88-2004&amp;ctx_ver=Z39.88-2004&amp;rfr_id=info%3Asid%2Fzotero.org%3A2&amp;rft_id=info%3Adoi%2F10.1093%2Fmnras%2Fstw2162&amp;rft_val_fmt=info%3Aofi%2Ffmt%3Akev%3Amtx%3Ajournal&amp;rft.genre=article&amp;rft.atitle=Chemical%20tagging%20with%20APOGEE%3A%20discovery%20of%20a%20large%20population%20of%20N-rich%20stars%20in%20the%20inner%20Galaxy&amp;rft.jtitle=Monthly%20Notices%20of%20the%20Royal%20Astronomical%20Society&amp;rft.stitle=Mon%20Not%20R%20Astron%20Soc&amp;rft.volume=465&amp;rft.issue=1&amp;rft.aufirst=Ricardo%20P.&amp;rft.aulast=Schiavon&amp;rft.au=Ricardo%20P.%20Schiavon&amp;rft.au=Olga%20Zamora&amp;rft.au=Ricardo%20Carrera&amp;rft.au=Sara%20Lucatello&amp;rft.au=A.%20C.%20Robin&amp;rft.au=Melissa%20Ness&amp;rft.au=Sarah%20L.%20Martell&amp;rft.au=Verne%20V.%20Smith&amp;rft.au=D.%20A.%20Garc%C3%ADa-Hern%C3%A1ndez&amp;rft.au=Arturo%20Manchado&amp;rft.au=Ralph%20Sch%C3%B6nrich&amp;rft.au=Nate%20Bastian&amp;rft.au=Cristina%20Chiappini&amp;rft.au=Matthew%20Shetrone&amp;rft.au=J.%20Ted%20Mackereth&amp;rft.au=Rob%20A.%20Williams&amp;rft.au=Szabolcs%20M%C3%A9sz%C3%A1ros&amp;rft.au=Carlos%20Allende%20Prieto&amp;rft.au=Friedrich%20Anders&amp;rft.au=Dmitry%20Bizyaev&amp;rft.au=Timothy%20C.%20Beers&amp;rft.au=S.%20Drew%20Chojnowski&amp;rft.au=Katia%20Cunha&amp;rft.au=Courtney%20Epstein&amp;rft.au=Peter%20M.%20Frinchaboy&amp;rft.au=Garc%C3%ADa%20P%C3%A9rez&amp;rft.au=Ana%20E&amp;rft.au=Fred%20R.%20Hearty&amp;rft.au=Jon%20A.%20Holtzman&amp;rft.au=Jennifer%20A.%20Johnson&amp;rft.au=Karen%20Kinemuchi&amp;rft.au=Steven%20R.%20Majewski&amp;rft.au=Demitri%20Muna&amp;rft.au=David%20L.%20Nidever&amp;rft.au=Duy%20Cuong%20Nguyen&amp;rft.au=Robert%20W.%20O'Connell&amp;rft.au=Daniel%20Oravetz&amp;rft.au=Kaike%20Pan&amp;rft.au=Marc%20Pinsonneault&amp;rft.au=Donald%20P.%20Schneider&amp;rft.au=Matthias%20Schultheis&amp;rft.au=Audrey%20Simmons&amp;rft.au=Michael%20F.%20Skrutskie&amp;rft.au=Jennifer%20Sobeck&amp;rft.au=John%20C.%20Wilson&amp;rft.au=Gail%20Zasowski&amp;rft.date=2017-02-11&amp;rft.pages=501-524&amp;rft.spage=501&amp;rft.epage=524&amp;rft.issn=0035-8711&amp;rft.language=en"></span>
  <div class="csl-entry" style="margin-bottom: 1em;">Serenelli, A., Johnson, J., Huber, D., Pinsonneault, M., Ball, W. H., Tayar, J., Aguirre, V. S., Basu, S., Troup, N., Hekker, S., Kallinger, T., Stello, D., Davies, G. R., Lund, M. N., Mathur, S., Mosser, B., Stassun, K. G., Chaplin, W. J., Yvonne Elsworth, García, R. A., Handberg, R., Holtzman, J., Hearty, F., García-Hernández, D. A., Patrick Gaulme and Zamora, O. (2017) ‘The First APOKASC Catalog of Kepler Dwarf and Subgiant Stars’, <i>The Astrophysical Journal Supplement Series</i>, vol. 233, no. 2, p. 23 [Online]. DOI: <a href="https://doi.org/10.3847/1538-4365/aa97df">10.3847/1538-4365/aa97df</a>.</div>
  <span class="Z3988" title="url_ver=Z39.88-2004&amp;ctx_ver=Z39.88-2004&amp;rfr_id=info%3Asid%2Fzotero.org%3A2&amp;rft_id=info%3Adoi%2F10.3847%2F1538-4365%2Faa97df&amp;rft_val_fmt=info%3Aofi%2Ffmt%3Akev%3Amtx%3Ajournal&amp;rft.genre=article&amp;rft.atitle=The%20First%20APOKASC%20Catalog%20of%20Kepler%20Dwarf%20and%20Subgiant%20Stars&amp;rft.jtitle=The%20Astrophysical%20Journal%20Supplement%20Series&amp;rft.stitle=ApJS&amp;rft.volume=233&amp;rft.issue=2&amp;rft.aufirst=Aldo&amp;rft.aulast=Serenelli&amp;rft.au=Aldo%20Serenelli&amp;rft.au=Jennifer%20Johnson&amp;rft.au=Daniel%20Huber&amp;rft.au=Marc%20Pinsonneault&amp;rft.au=Warrick%20H.%20Ball&amp;rft.au=Jamie%20Tayar&amp;rft.au=Victor%20Silva%20Aguirre&amp;rft.au=Sarbani%20Basu&amp;rft.au=Nicholas%20Troup&amp;rft.au=Saskia%20Hekker&amp;rft.au=Thomas%20Kallinger&amp;rft.au=Dennis%20Stello&amp;rft.au=Guy%20R.%20Davies&amp;rft.au=Mikkel%20N.%20Lund&amp;rft.au=Savita%20Mathur&amp;rft.au=Benoit%20Mosser&amp;rft.au=Keivan%20G.%20Stassun&amp;rft.au=William%20J.%20Chaplin&amp;rft.au=undefined&amp;rft.au=Rafael%20A.%20Garc%C3%ADa&amp;rft.au=Rasmus%20Handberg&amp;rft.au=Jon%20Holtzman&amp;rft.au=Fred%20Hearty&amp;rft.au=D.%20A.%20Garc%C3%ADa-Hern%C3%A1ndez&amp;rft.au=undefined&amp;rft.au=Olga%20Zamora&amp;rft.date=2017&amp;rft.pages=23&amp;rft.issn=0067-0049&amp;rft.language=en"></span>
  <div class="csl-entry" style="margin-bottom: 1em;">Shukla, N. (2018) <i>Machine Learning with TensorFlow</i>, 1 edition., Shelter Island, NY, Manning Publications.</div>
  <span class="Z3988" title="url_ver=Z39.88-2004&amp;ctx_ver=Z39.88-2004&amp;rfr_id=info%3Asid%2Fzotero.org%3A2&amp;rft_id=urn%3Aisbn%3A978-1-61729-387-0&amp;rft_val_fmt=info%3Aofi%2Ffmt%3Akev%3Amtx%3Abook&amp;rft.genre=book&amp;rft.btitle=Machine%20Learning%20with%20TensorFlow&amp;rft.place=Shelter%20Island%2C%20NY&amp;rft.publisher=Manning%20Publications&amp;rft.edition=1%20edition&amp;rft.aufirst=Nishant&amp;rft.aulast=Shukla&amp;rft.au=Nishant%20Shukla&amp;rft.date=2018-02-12&amp;rft.tpages=272&amp;rft.isbn=978-1-61729-387-0&amp;rft.language=English"></span>
  <div class="csl-entry" style="margin-bottom: 1em;">Singh, H. P., Gulati, R. K. and Gupta, R. (1998) ‘Stellar spectral classification using principal component analysis and artificial neural networks’, <i>Monthly Notices of the Royal Astronomical Society</i>, vol. 295, no. 2, pp. 312–318 [Online]. DOI: <a href="https://doi.org/10.1046/j.1365-8711.1998.01255.x">10.1046/j.1365-8711.1998.01255.x</a>.</div>
  <span class="Z3988" title="url_ver=Z39.88-2004&amp;ctx_ver=Z39.88-2004&amp;rfr_id=info%3Asid%2Fzotero.org%3A2&amp;rft_id=info%3Adoi%2F10.1046%2Fj.1365-8711.1998.01255.x&amp;rft_val_fmt=info%3Aofi%2Ffmt%3Akev%3Amtx%3Ajournal&amp;rft.genre=article&amp;rft.atitle=Stellar%20spectral%20classification%20using%20principal%20component%20analysis%20and%20artificial%20neural%20networks&amp;rft.jtitle=Monthly%20Notices%20of%20the%20Royal%20Astronomical%20Society&amp;rft.stitle=Mon%20Not%20R%20Astron%20Soc&amp;rft.volume=295&amp;rft.issue=2&amp;rft.aufirst=Harinder%20P.&amp;rft.aulast=Singh&amp;rft.au=Harinder%20P.%20Singh&amp;rft.au=Ravi%20K.%20Gulati&amp;rft.au=Ranjan%20Gupta&amp;rft.date=1998-04-01&amp;rft.pages=312-318&amp;rft.spage=312&amp;rft.epage=318&amp;rft.issn=0035-8711&amp;rft.language=en"></span>
  <div class="csl-entry" style="margin-bottom: 1em;">Thanaki, J. (2018) <i>Machine Learning Solutions</i>, S.l., Packt Publishing.</div>
  <span class="Z3988" title="url_ver=Z39.88-2004&amp;ctx_ver=Z39.88-2004&amp;rfr_id=info%3Asid%2Fzotero.org%3A2&amp;rft_id=urn%3Aisbn%3A978-1-78839-004-0&amp;rft_val_fmt=info%3Aofi%2Ffmt%3Akev%3Amtx%3Abook&amp;rft.genre=book&amp;rft.btitle=Machine%20Learning%20Solutions&amp;rft.place=S.l.&amp;rft.publisher=Packt%20Publishing&amp;rft.aufirst=Jalaj&amp;rft.aulast=Thanaki&amp;rft.au=Jalaj%20Thanaki&amp;rft.date=2018-04-26&amp;rft.tpages=566&amp;rft.isbn=978-1-78839-004-0&amp;rft.language=English"></span>
  <div class="csl-entry" style="margin-bottom: 1em;">Toomey, D. (2017) <i>Jupyter for Data Science: Exploratory analysis, statistical modeling, machine learning, and data visualization with Jupyter</i>, Birmingham Mumbai, Packt Publishing.</div>
  <span class="Z3988" title="url_ver=Z39.88-2004&amp;ctx_ver=Z39.88-2004&amp;rfr_id=info%3Asid%2Fzotero.org%3A2&amp;rft_id=urn%3Aisbn%3A978-1-78588-007-0&amp;rft_val_fmt=info%3Aofi%2Ffmt%3Akev%3Amtx%3Abook&amp;rft.genre=book&amp;rft.btitle=Jupyter%20for%20Data%20Science%3A%20Exploratory%20analysis%2C%20statistical%20modeling%2C%20machine%20learning%2C%20and%20data%20visualization%20with%20Jupyter&amp;rft.place=Birmingham%20Mumbai&amp;rft.publisher=Packt%20Publishing&amp;rft.aufirst=Dan&amp;rft.aulast=Toomey&amp;rft.au=Dan%20Toomey&amp;rft.date=2017-10-20&amp;rft.tpages=242&amp;rft.isbn=978-1-78588-007-0&amp;rft.language=English"></span>
  <div class="csl-entry" style="margin-bottom: 1em;">Van de Ven, B. and Paprocki, M. (2018) <i>bokeh: Interactive Web Plotting for Python</i>, Python, Bokeh [Online]. Available at <a href="https://github.com/bokeh/bokeh">https://github.com/bokeh/bokeh</a> (Accessed 15 June 2018).</div>
  <span class="Z3988" title="url_ver=Z39.88-2004&amp;ctx_ver=Z39.88-2004&amp;rfr_id=info%3Asid%2Fzotero.org%3A2&amp;rft_val_fmt=info%3Aofi%2Ffmt%3Akev%3Amtx%3Adc&amp;rft.type=computerProgram&amp;rft.title=bokeh%3A%20Interactive%20Web%20Plotting%20for%20Python&amp;rft.rights=BSD-3-Clause&amp;rft.identifier=https%3A%2F%2Fgithub.com%2Fbokeh%2Fbokeh&amp;rft.aufirst=Bryan&amp;rft.aulast=Van%20de%20Ven&amp;rft.au=Bryan%20Van%20de%20Ven&amp;rft.au=Mateusz%20Paprocki&amp;rft.date=2018-06-15"></span>
  <div class="csl-entry" style="margin-bottom: 1em;">Wang, K., Guo, P. and Luo, A.-L. (2017) ‘A new automated spectral feature extraction method and its application in spectral classification and defective spectra recovery’, <i>Monthly Notices of the Royal Astronomical Society</i>, vol. 465, no. 4, pp. 4311–4324 [Online]. DOI: <a href="https://doi.org/10.1093/mnras/stw2894">10.1093/mnras/stw2894</a>.</div>
  <span class="Z3988" title="url_ver=Z39.88-2004&amp;ctx_ver=Z39.88-2004&amp;rfr_id=info%3Asid%2Fzotero.org%3A2&amp;rft_id=info%3Adoi%2F10.1093%2Fmnras%2Fstw2894&amp;rft_val_fmt=info%3Aofi%2Ffmt%3Akev%3Amtx%3Ajournal&amp;rft.genre=article&amp;rft.atitle=A%20new%20automated%20spectral%20feature%20extraction%20method%20and%20its%20application%20in%20spectral%20classification%20and%20defective%20spectra%20recovery&amp;rft.jtitle=Monthly%20Notices%20of%20the%20Royal%20Astronomical%20Society&amp;rft.stitle=Mon%20Not%20R%20Astron%20Soc&amp;rft.volume=465&amp;rft.issue=4&amp;rft.aufirst=Ke&amp;rft.aulast=Wang&amp;rft.au=Ke%20Wang&amp;rft.au=Ping%20Guo&amp;rft.au=A.-Li%20Luo&amp;rft.date=2017-03-11&amp;rft.pages=4311-4324&amp;rft.spage=4311&amp;rft.epage=4324&amp;rft.issn=0035-8711&amp;rft.language=en"></span>
  <div class="csl-entry" style="margin-bottom: 1em;">Yu, A., Chung, C. and Yim, A. (2017) <i>Matplotlib 2.x By Example: Multi-dimensional charts, graphs, and plots in Python</i>, Packt Publishing - ebooks Account.</div>
  <span class="Z3988" title="url_ver=Z39.88-2004&amp;ctx_ver=Z39.88-2004&amp;rfr_id=info%3Asid%2Fzotero.org%3A2&amp;rft_id=urn%3Aisbn%3A978-1-78829-526-0&amp;rft_val_fmt=info%3Aofi%2Ffmt%3Akev%3Amtx%3Abook&amp;rft.genre=book&amp;rft.btitle=Matplotlib%202.x%20By%20Example%3A%20Multi-dimensional%20charts%2C%20graphs%2C%20and%20plots%20in%20Python&amp;rft.publisher=Packt%20Publishing%20-%20ebooks%20Account&amp;rft.aufirst=Allen&amp;rft.aulast=Yu&amp;rft.au=Allen%20Yu&amp;rft.au=Claire%20Chung&amp;rft.au=Aldrin%20Yim&amp;rft.date=2017-08-28&amp;rft.tpages=334&amp;rft.isbn=978-1-78829-526-0&amp;rft.language=English"></span>
  <div class="csl-entry" style="margin-bottom: 1em;">Zaccone, G., Karim, M. R. and Menshawy, A. (2017) <i>Deep Learning with TensorFlow: Explore neural networks with Python</i>, Birmingham Mumbai, Packt Publishing.</div>
  <span class="Z3988" title="url_ver=Z39.88-2004&amp;ctx_ver=Z39.88-2004&amp;rfr_id=info%3Asid%2Fzotero.org%3A2&amp;rft_id=urn%3Aisbn%3A978-1-78646-978-6&amp;rft_val_fmt=info%3Aofi%2Ffmt%3Akev%3Amtx%3Abook&amp;rft.genre=book&amp;rft.btitle=Deep%20Learning%20with%20TensorFlow%3A%20Explore%20neural%20networks%20with%20Python&amp;rft.place=Birmingham%20Mumbai&amp;rft.publisher=Packt%20Publishing&amp;rft.aufirst=Giancarlo&amp;rft.aulast=Zaccone&amp;rft.au=Giancarlo%20Zaccone&amp;rft.au=Md%20Rezaul%20Karim&amp;rft.au=Ahmed%20Menshawy&amp;rft.date=2017-04-24&amp;rft.tpages=320&amp;rft.isbn=978-1-78646-978-6&amp;rft.language=English"></span>
  <div class="csl-entry">Zocca, V., Spacagna, G., Slater, D. and Roelants, P. (2017) <i>Python Deep Learning</i>, Birmingham Mumbai, Packt Publishing.</div>
  <span class="Z3988" title="url_ver=Z39.88-2004&amp;ctx_ver=Z39.88-2004&amp;rfr_id=info%3Asid%2Fzotero.org%3A2&amp;rft_id=urn%3Aisbn%3A978-1-78646-445-3&amp;rft_val_fmt=info%3Aofi%2Ffmt%3Akev%3Amtx%3Abook&amp;rft.genre=book&amp;rft.btitle=Python%20Deep%20Learning&amp;rft.place=Birmingham%20Mumbai&amp;rft.publisher=Packt%20Publishing&amp;rft.aufirst=Valentino&amp;rft.aulast=Zocca&amp;rft.au=Valentino%20Zocca&amp;rft.au=Gianmario%20Spacagna&amp;rft.au=Daniel%20Slater&amp;rft.au=Peter%20Roelants&amp;rft.date=2017-04-28&amp;rft.tpages=406&amp;rft.isbn=978-1-78646-445-3&amp;rft.language=English"></span>
</div>

# Reading Notes

## Notes on Fabbro _et al_.

* This are some brief notes on:
 * Fabbro, S., Venn, K., O’Briain, T., Bialek, S., Kielty, C., Jahandar, F. and Monty, S. (2018) ‘An Application of Deep Neural Networks in the Analysis of Stellar Spectra’, <i>Monthly Notices of the Royal Astronomical Society</i>, vol. 475, no. 3, pp. 2978–2993 [Online]. DOI: <a href="https://doi.org/10.1093/mnras/stx3298">10.1093/mnras/stx3298</a>.
* Read in conjunction with C.L. Kielty (4th author on Fabbro _et al_.), Master of Science thesis (Kielty, 2017), see additional ML refs below.
* Training and optimisation
 * "When StarNet is trained on the ASPCAP stellar parameters, intrinsic errors in the ASPCAP pipeline are implicitly propagated throughout the results".
 * "StarNet performs poorly on small training sets"
 * "Deep learning architectures, such as StarNet, typically involve experimentation and tuning of hyper-parameters neither related to the physical model or the NN model, in order to converge to an effective model."
 * hyperparameter optimization - see Kielty
 * "Deep learning architectures, such as StarNet, typically involve experimentation and tuning of hyper-parameters in order to converge to an adequate model." (Fabbro)
 * Parameter normalisation to achieve equal weighting
 * "He Normal initializer and ADAM optimizer allow for a deeper model to reach convergence, while the ReLU activation and max pooling layer simplify the model to an optimal rank of complexity". (Kielty)
* Stellar parameters derived:
 * $T_{eff}$, $\log(g)$, [Fe/H]
 * (why not also e.g. [$\alpha$/H], [M/H] or individual C, N or other abundances?)
 * (Note: linear relation of microturbulence to gravity assumed in ASPCAP [García Pérez _et al_])
* Training convergence time for weights depends on size of input and model architecture (see performance notes below)
* A good deal of experimentation was required with architecture and hyper-parameters (Kielty)
* 224,000 synthetic training spectra plus 36,000 synthetic reference set
* Cross-validation using reference set after each training iteration in initial experimentation
* StarNet cannot weight according to S/N of input spectrum, but synthetic noisy spectra used in training
* After training only takes ~1 minute to process 143k spectra of APOGEE DR13


In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
Notebook code is hidden, click <a href="javascript:code_toggle()">here</a> to toggle''')

